In [1]:
import imp
import time
import numpy as np
from tqdm import tqdm
from scipy.io import loadmat
import rsatoolbox as rsa_old # not sure what convention is
rsa_new = imp.load_source('rsatoolbox','/home/smazurchuk/Desktop/tmp/rsatoolbox/rsatoolbox/rsatoolbox/__init__.py')

In [2]:
# Load dataset
dsets = np.random.rand(18,1800,29696)
lh_rois = np.array([np.random.choice(29696,100,replace=False) for _ in range(10)]) # Searchlight vertices

# Generate Word and presentation numbers
words = []; presentation = [];
for i in range(1800):
    words.append(f'_{int(i/6)}_')
    presentation.append(np.mod(i,6))

In [5]:
ds = [];
for i in range(len(dsets)):
    tmp    = dsets[i][:,:29696]
    tmp_ds = rsa_old.data.dataset.Dataset(measurements = tmp,
             descriptors = {'subj':i},
             obs_descriptors = {'words':words, 'presentation':presentation},
             channel_descriptors={'vertices':np.arange(29696)})
    ds.append(tmp_ds)

In [11]:
# Previous
def lh_searchlight_old(i):
    neighbors = lh_rois[i]
    rdms_data = []
    for j in range(len(dsets)): # Iterate over subjects
        sdata = ds[j].subset_channel('vertices',neighbors)
        tmp = rsa_old.rdm.calc_rdm(sdata, descriptor='words',method='crossnobis',
                                cv_descriptor='presentation')
        rdms_data.append(tmp)
    rdms_data = rsa_old.rdm.rdms.concat(rdms_data)
    return rdms_data
# New
def lh_searchlight_new(i):
    neighbors = lh_rois[i]
    rdms_data = []
    for j in range(len(dsets)): # Iterate over subjects
        sdata = ds[j].subset_channel('vertices',neighbors)
        tmp = rsa_new.rdm.calc_rdm(sdata, descriptor='words',method='crossnobis',
                                cv_descriptor='presentation')
        rdms_data.append(tmp)
    rdms_data = rsa_new.rdm.rdms.concat(rdms_data)
    return rdms_data

In [12]:
start = time.time()
tmp = [lh_searchlight_old(i) for i in range(10)]
print(f'Elapsed Time:{time.time() - start:.1f} seconds')

Elapsed Time:126.3 seconds


In [13]:
start = time.time()
tmp = [lh_searchlight_new(i) for i in range(10)]
print(f'Elapsed Time:{time.time() - start:.1f} seconds')

Elapsed Time:127.6 seconds


In [14]:
def lh_searchlight_new2(i):
    rdms_data = []; 
    neighbors = np.nonzero(lh_rois[i])[0]
    for j in range(len(dsets)):
        ds = rsa_new.data.dataset.Dataset(measurements = dsets[j][:,:29696][:,neighbors],
                 descriptors = {'subj':i, 'center':j},
                 obs_descriptors = {'words':words, 'presentation':presentation})
        tmp = rsa_new.rdm.calc_rdm(ds, descriptor='words',method='crossnobis',
                                cv_descriptor='presentation')
        rdms_data.append(tmp)
    return rdms_data

In [15]:
start = time.time()
tmp = [lh_searchlight_new2(i) for i in range(10)]
print(f'Elapsed Time:{time.time() - start:.1f} seconds')

Elapsed Time:111.3 seconds
